# Clustering

 Within this notebook we will play with clustering.
 
 * First try the hierarchical clustering and k-means on artificial data to see visually how those two methods work.
 * Then, on the Iris dataset we use the hierarchical clustering to observe its structure.
 * Finally, we focus on the vector quantization as a nice application of k-means algorithm to compress images.

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

np.set_printoptions(precision=5, suppress=True)  # suppress scientific float notation (so 0.000 is printed as 0.)

## Artificial data - hierarchical clustering & k-means

### Artificial data generation

We generate random samples from three different multivariate normal distributions and merge them.

The parameters of multivariate normal distribution are:
* The mean - corresponding to the center of the cluster.
* The covariance matrix - corresponding to the shape (circle or ellipse with some direction) and size of the cluster.

In [ ]:
# generate three clusters: a with 60 points, b with 40, c with 20:
np.random.seed(50)  # for repeatability of this tutorial
a = np.random.multivariate_normal([7, 7], [[2, 0.5], [0.5, 2]], size=[60,])
b = np.random.multivariate_normal([0, 15], [[2, 0], [0, 2]], size=[40,])
c = np.random.multivariate_normal([15, 0], [[3, 1], [1, 4]], size=[20,])

# merge the clusters into X
X = np.concatenate((a, b, c),)

# print the shape
print(X.shape)

# visualize the data
plt.scatter(X[:,0], X[:,1])
plt.axis('equal')
plt.show()

### Hierarchical agglomerative clustering

We use the `scipy` library and especially its hierarchical clustering functions. [(docs here)](https://docs.scipy.org/doc/scipy/reference/cluster.hierarchy.html).

This part is inspired by [this blog post](https://joernhees.de/blog/2015/08/26/scipy-hierarchical-clustering-and-dendrogram-tutorial/).

#### First we create the dendrogram
We use a `linkage` function for that. [(docs here)](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html#scipy.cluster.hierarchy.linkage).
* One of its important arguments is a method how the cluster distance is measured.
* First we use single linkage, but later you can play with other ones - complete linkage / average linkage / ward's method.

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage

# generate the linkage matrix 
Z = linkage(X, 'single')

# see the shape of the output
print(Z.shape)

The result is a linkage matrix where each row corresponds to a merging of two clusters. In columns we have:
* the index of a first merged cluster
* the index of the second merged cluster
* the distance between merged clusters
* the number of original observations in the newly formed cluster


In [ ]:
# Observe first 5 rows of Z
print(Z[:5,:])

# especially focus on the 3rd row where the second cluster with index 120 is actually the first one created by merging.

#### Now let us visualize the dendrogram
For this we can use the `dendrogram` function. [(docs here)](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.dendrogram.html#scipy.cluster.hierarchy.dendrogram)

In [ ]:
# calculate full dendrogram
plt.figure(figsize=(25, 10))
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('sample index')
plt.ylabel('distance')
dendrogram(
    Z,
    leaf_rotation=90.,  # rotates the x axis labels
    leaf_font_size=8.,  # font size for the x axis labels
)
plt.show()

We can see that:
* horizontal lines are cluster merges
* vertical lines tell us which clusters/labels were part of merge forming that new cluster
* heights of the horizontal lines corresponds to distances between merged clusters

In [ ]:
# We can also plot just the upper part of the dendrogram
plt.title('Hierarchical Clustering Dendrogram (truncated)')
plt.xlabel('sample index or (cluster size)')
plt.ylabel('distance')
dendrogram(
    Z,
    truncate_mode='lastp',  # show only the last p merged clusters
    p=10,  # show only the last p merged clusters
    leaf_rotation=90.,
    leaf_font_size=12.,
    show_contracted=True,  # to get a distribution impression in truncated branches
)
plt.show()

#### More fancy version of dendrogram plot
With [annotating the distances inside the dendrogram](https://stackoverflow.com/questions/11917779/how-to-plot-and-annotate-hierarchical-clustering-dendrograms-in-scipy-matplotlib/12311618#12311618)

In [ ]:
def fancy_dendrogram(*args, **kwargs):
    max_d = kwargs.pop('max_d', None)
    if max_d and 'color_threshold' not in kwargs:
        kwargs['color_threshold'] = max_d
    annotate_above = kwargs.pop('annotate_above', 0)

    ddata = dendrogram(*args, **kwargs)

    if not kwargs.get('no_plot', False):
        plt.title('Hierarchical Clustering Dendrogram (truncated)')
        plt.xlabel('sample index or (cluster size)')
        plt.ylabel('distance')
        for i, d, c in zip(ddata['icoord'], ddata['dcoord'], ddata['color_list']):
            x = 0.5 * sum(i[1:3])
            y = d[1]
            if y > annotate_above:
                plt.plot(x, y, 'o', c=c)
                plt.annotate("%.3g" % y, (x, y), xytext=(0, -5),
                             textcoords='offset points',
                             va='top', ha='center')
        if max_d:
            plt.axhline(y=max_d, c='k')
    return ddata

In [ ]:
fancy_dendrogram(
    Z,
    truncate_mode='lastp',
    p=12,
    leaf_rotation=90.,
    leaf_font_size=12.,
    show_contracted=True,
    annotate_above=1.9,  # useful in small plots so annotations don't overlap
)
plt.show()

## Retreiving clusters from the dendrogram
For this we have the `fcluster` function. [(docs here)](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.fcluster.html#scipy.cluster.hierarchy.fcluster)

#### If we know the cut height 
i.e. the threshold for the cluster distance

In [ ]:
from scipy.cluster.hierarchy import fcluster
max_d = 5
clusters = fcluster(Z, max_d, criterion='distance')
# show clusters
print(clusters)

# figure
plt.scatter(X[:,0], X[:,1], c=clusters, cmap='brg')  # plot points with cluster dependent colors
plt.show()

In [ ]:
# show the cut in the truncated dendrogram
fancy_dendrogram(
    Z,
    truncate_mode='lastp',
    p=12,
    leaf_rotation=90.,
    leaf_font_size=12.,
    show_contracted=True,
    annotate_above=1.9,  # useful in small plots so annotations don't overlap
    max_d = max_d,
)
plt.show()

#### If we know the number of clusters $k$

In [ ]:
k = 3
clusters = fcluster(Z, k, criterion='maxclust')
# show clusters
print(clusters)

# figure
plt.scatter(X[:,0], X[:,1], c=clusters, cmap='brg')  # plot points with cluster dependent colors
plt.show()

### K-means algorithm
We use `clustering` part of the `sklearn` library. [(docs here)](http://scikit-learn.org/stable/modules/clustering.html#k-means)

There is a function `Kmeans` which can be used for that. [(docs here)](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans)

In [ ]:
from sklearn.cluster import KMeans

# first lets try 2 clusters
k = 2
kmeans = KMeans(n_clusters = k, random_state = 1).fit(X)

# to show resulting clusters
print(kmeans.labels_)
# to see cluster centers
print(kmeans.cluster_centers_)

# figure
plt.scatter(X[:,0], X[:,1], c=kmeans.labels_, cmap='brg', alpha=0.4)  # plot points with cluster dependent colors
plt.scatter(kmeans.cluster_centers_[:,0], kmeans.cluster_centers_[:,1], c = 'black', s=100)
plt.show()

In [ ]:
# the same for 3 clusters
k = 3
kmeans = KMeans(n_clusters = k, random_state = 1).fit(X)

# figure
plt.scatter(X[:,0], X[:,1], c=kmeans.labels_, cmap='brg', alpha=0.4)  # plot points with cluster dependent colors
plt.scatter(kmeans.cluster_centers_[:,0], kmeans.cluster_centers_[:,1], c = 'black', s=100)
plt.show()

In the default setting the algorithm initializes in some smart way, `init = 'k-means++'` [David, Vassilvitskii (2007)](http://ilpubs.stanford.edu:8090/778/1/2006-13.pdf), and the whole run is repeated 10 times, `n_init = 10`.

It is also possible to initialize it manually or with random selection from the data.

In [ ]:
k = 3
# manual initialization
initial_centers = np.array([[0,10],[10,10],[10,0]])

# clusterring
kmeans = KMeans(n_clusters = k, random_state = 1, init = initial_centers, n_init = 1).fit(X)

# figure
plt.scatter(X[:,0], X[:,1], c=kmeans.labels_, cmap='brg', alpha=0.4)  # plot points with cluster dependent colors
plt.scatter(initial_centers[:,0], initial_centers[:,1], c = 'black', s=50, alpha = 0.9, marker = 'x') # initial centers
plt.scatter(kmeans.cluster_centers_[:,0], kmeans.cluster_centers_[:,1], c = 'black', s=100) # final centers
plt.legend(['Data', 'Initial Centers', 'Final Centroids'])
plt.show()

### The $k$-means may be also performed manually

In [ ]:
from sklearn.metrics import pairwise_distances_argmin
centers = initial_centers
for i in range(3):
    y_pred = pairwise_distances_argmin(X, centers)
    new_centers = np.array([X[y_pred == i].mean(0) for i in range(len(centers))])

    # figure
    plt.scatter(X[:,0], X[:,1], c=y_pred, cmap='brg', alpha=0.4)  # plot points with cluster dependent colors
    plt.scatter(centers[:,0], centers[:,1], c = 'black', s=50, alpha = 0.9, marker = 'x') # old centers
    plt.scatter(new_centers[:,0], new_centers[:,1], c = 'black', s=100) # new centers
    plt.title('Manual $k$-means, step ' + str(i+1))
    plt.legend(['Data', 'Old Centers', 'New Centers'])
    plt.show()
    
    centers = new_centers

### Elbow method for $k$ estimation

In [ ]:
ix = np.zeros(5)
iy = np.zeros(5)
for k in range(ix.shape[0]):
    kmeans = KMeans(n_clusters=k+1, random_state = 1)
    kmeans.fit(X)
    iy[k] = kmeans.inertia_
    ix[k] = k+1

plt.xlabel('$k$')
plt.ylabel('Objective function')
plt.plot(ix, iy, 'o-')
plt.show()

## Task 1 - perform hierarchical clustering on Iris dataset
* Search for 3 clusters
* Discuss and measure somehow the quality of final clusters. The real types of irises’ (Setosa, Versicolour, and Virginica) are stored in y variable.
* Try to find a cluster distance function such that the final clusters correspond to real types as accurate as possible.

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data
y = iris.target

print('X shape:', X.shape)

In [ ]:
## your code here

Z = linkage(X, 'single') # best is ward - at least between the triple - single, complete and ward
k = 3
clusters = fcluster(Z, k, criterion='maxclust')
# recode values so they match - this have to be done manually
print('Assigned clusters:', clusters)
print('True types:', y)
# - ward method & single linkage
clusters[clusters == 1] = 0
clusters[clusters == 3] = 1

# - complete linkage
# clusters[clusters == 3] = 0
# clusters[clusters == 2] = 3
# clusters[clusters == 1] = 2
# clusters[clusters == 3] = 1

# show accuracy
print('Accuracy:', 1 - np.abs(clusters - y).sum()/y.shape[0])

### Task 2 - perform vector quantization with $k$-means on a figure
You need to install Pillow package
`pip install Pillow`. [(docs here)](https://pillow.readthedocs.io/en/5.3.x/index.html)

First some code that brings you to the task itself.

In [ ]:
from PIL import Image
# open and conver to grayscale
im = Image.open("figure.jpg").convert("L")
# covert to numpy array of numbers between 0 and 1
pix = np.array(im)/255.0
print('Shape of the array:', pix.shape)
# visualize
plt.imshow(pix, cmap="gray", clim=(0, 1));
plt.show()

Now the task:
* crop the image width to be a multiply of 4
* create column blocks of length 4 - i.e. subparts from the original image of the shape (1,4)
* perform k-means clusterring with k = 255 - i.e. one byte will be sufficient to transmit the cluster indices
* extract centroids and labels from the clustering
* decode them back to the array of the original shape - **hint** -  use: `restored = np.take(centroids, labels, axis = 0)`
* visualize the result
* discuss the size reduction (compression) when one uses the centroids and lables instead of the original pixels.

In [ ]:
## your code here


# set block length to 4
block_len = 4

# crop the image to optimal size
rows,cols = pix.shape
pix = pix[:, :(cols - cols % block_len)]

# reshape the figure to create the blocks
X = pix.reshape(-1, block_len)
print(X.shape)

# perform clusterring
k = 255 # i.e. one byte will be sufficient for transfering the values
kmeans = KMeans(n_clusters = k, random_state = 1, n_init = 2).fit(X)

# extract the centroids and labels
centroids = kmeans.cluster_centers_
labels = kmeans.labels_
print(centroids.shape)
print(labels.shape)

# restore the compressed image from centroids and labels
restored = np.take(centroids, labels, axis = 0)
final_pix = restored.reshape(pix.shape)

# visualize a result
plt.imshow(final_pix, cmap="gray", clim=(0, 1));
plt.show()